<a href="https://colab.research.google.com/github/JapiKredi/RAG_assignment_Research_papers/blob/main/RAG_assignment_Research_papers_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Assignment "Course 3 Mr.HelpMate"
### Learner: Jasper Bongers
#### Option 1: Build Your Own Project (BYOP)

- I a building a generative AI system that is reading 5 popular Research Papers in Natural Language Understanding.
- I am using multiple Large language Model Embeddings to translate the text into vector embeddings.
- I am using multiple Large Language Foundation modls. I am using OPen Ai, LLaMa2 and other models as well.
- I am using multiple LLM transformer-based models in order to process the research papers and finetune the foundation language models.

# <font color = green> Solution 1


## 1. <font color = red> Install and Import the Required Libraries

Please run the following code on a GPU, as running it on a CPU can create a crash

In [1]:
!pip -q install langchain openai tiktoken chromadb pypdf sentence_transformers InstructorEmbedding

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.9/283.9 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.3/230.3 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 3.4 MB/s 

In [2]:
!pip install accelerate
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.2 MB/s eta 0:00:00


In [28]:
import accelerate
import bitsandbytes

In [4]:
# Import all the required Libraries
import openai
import pypdf
from pathlib import Path
import pandas as pd
from operator import itemgetter
import json
import tiktoken
import chromadb

In [5]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings
from langchain.embeddings import HuggingFaceInstructEmbeddings
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
from sentence_transformers import CrossEncoder, util

In [6]:
!pip show langchain

Name: langchain
Version: 0.1.4
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, jsonpatch, langchain-community, langchain-core, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


## 2. <font color = red> Read, Process, and Chunk the PDF Files

In [7]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [8]:
# Set the API key
filepath = "/content/drive/My Drive/GenerativeAI/MateAI/"

with open(filepath + "Jasper_OpenAI_API_Key.txt", "r") as f:
  openai.api_key = ' '.join(f.readlines())

In [9]:
import os
os.environ["OPENAI_API_KEY"] = openai.api_key

## Load multiple and process documents

In [11]:
# Load and process the text files
loader = DirectoryLoader("/content/drive/My Drive/GenerativeAI/MateAI/rag_assignment/research_articles/", glob="./*.pdf", loader_cls=PyPDFLoader)
documents = loader.load()

In [12]:
len(documents)

346

## Splitting, Chunking of text

In [13]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [14]:
len(texts)

1419

In [15]:
texts[10]

Document(page_content='1024) since we break batches into multiple updates. See Table 1 in the appendix for exact numbers.\nNonoverlapping Inference To train on or evaluate a sequence longer than Ltokens, it is typical\nto segment the sequence into L-length subsequences and train on or evaluate them independently.\nUnless otherwise stated, we use nonoverlapping inference to report perplexity scores.\nExtrapolation During Inference Formally, the functions that deﬁne a transformer layer are ag-\nnostic to input length;3they map from some arbitrary, unﬁxed number of input vectors to the same\nnumber of output vectors. When transformers are applied to data that is inherently sequential, like\ntext, positional information is injected into the inputs in various ways.\nVaswani et al. (2017) discussed two options for embedding positions into vectors to be added to word\nembeddings: learning embeddings for speciﬁc positions and unlearned sinusoidal embeddings. They', metadata={'source': '/conten

## Embedding the text via regular HuggingFace process

In [14]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
                                                      model_kwargs={"device": "cuda"})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.40k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer
max_seq_length  512


### Creating Chroma Vector Database

In [15]:
# Define the path where chroma collections will be stored
chroma_data_path = '/content/drive/My Drive/GenerativeAI/MateAI/rag_assignment/ChromaDB_Data'

In [16]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = chroma_data_path

## Here is the nmew embeddings being used
embedding = instructor_embeddings

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [17]:
# persiste the db to disk
vectordb.persist()
vectordb = None

In [18]:
# Now we can load the persisted database from disk, and use it as normal.
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding)

## 4. <font color = red> Semantic Search

In this section, we will perform a semantic search of a query in the collections embeddings to get several top semantically similar results.

In [19]:
## Make a retriever
retriever = vectordb.as_retriever(search_kwargs={"k": 1})

In [20]:
# Get a relevant query
query = "What is Flash attention?"

In [21]:
docs = retriever.get_relevant_documents("What is Flash attention?")

In [22]:
retriever.search_type

'similarity'

## Make a chain

In [23]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(),
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


## 6. Retrieval Augmented Generation

Now that we have the final top search results, we can pass it to an GPT 3.5 along with the user query and a well-engineered prompt, to generate a direct answer to the query along with citations, rather than returning whole pages/chunks.

In [24]:
## Cite sources

import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [25]:
# Querry 1: What is Flash attention?
query = "What is Flash attention?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




FlashAttention is a new attention algorithm that uses techniques such as tiling and storing the softmax
normalization factor to reduce memory accesses and improve performance in computing attention.


Sources:
/content/drive/My Drive/GenerativeAI/MateAI/rag_assignment/research_articles/Flash-attention.pdf


In [26]:
# Querry 2: What is self attention?
query = "What is self-attention?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Self-attention is a mechanism used in neural networks, specifically in Transformer models, to capture
relationships between different input tokens or elements. It involves computing a representation for each
input element by considering its relationship with all other elements in the input sequence. This allows the
model to learn dependencies and long-range relationships between elements, making it useful for tasks such as
natural language processing and sequence prediction. However, self-attention can be computationally expensive
and memory-intensive for long sequences, leading to the development of approximate attention methods.


Sources:
/content/drive/My Drive/GenerativeAI/MateAI/rag_assignment/research_articles/Flash-attention.pdf


In [27]:
# Querry 3: What is multi head attention?
query = "What is multi head attention?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Multi head attention is a computation used in deep learning models, specifically in Transformers, that is
highly memory-intensive. It involves touching the GPU memory multiple times and can be parallelizable across
multiple GPUs. The IO-aware approach aims to optimize this computation and potentially extend it to other
modules in the deep network.


Sources:
/content/drive/My Drive/GenerativeAI/MateAI/rag_assignment/research_articles/Flash-attention.pdf


In [28]:
# Querry 4: What does IO-aware mean?
query = "What does IO-aware mean?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 IO-aware means being conscious of input/output operations in order to optimize performance and efficiency.


Sources:
/content/drive/My Drive/GenerativeAI/MateAI/rag_assignment/research_articles/Flash-attention.pdf


In [29]:
# Querry 5: What is tiling in flash-attention?
query = "What is tiling in flash-attention?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Tiling in flash-attention is a technique used to reduce the amount of HBM accesses in the attention
algorithm. It involves breaking down the input matrix into smaller tiles and computing attention on each tile
separately, reducing the overall number of HBM accesses needed.


Sources:
/content/drive/My Drive/GenerativeAI/MateAI/rag_assignment/research_articles/Flash-attention.pdf


In [30]:
# Querry 6: What tools can be used with toolformer?
query = "What tools can be used with toolformer?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Toolformer can use external tools through simple APIs in a self-supervised way, requiring only a few
demonstrations for each API.


Sources:
/content/drive/My Drive/GenerativeAI/MateAI/rag_assignment/research_articles/toolformer.pdf


In [31]:
# Querry 7: What are the best retrieval augmentations for LLMs?
query = "What are the best retrieval augmentations for LLMs?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Dense and sparse retrievers.


Sources:
/content/drive/My Drive/GenerativeAI/MateAI/rag_assignment/research_articles/Augmenting LLMs Survey.pdf


## Deleting the Chroma Vector Database

In [ ]:
# Define the path where chroma collections will be stored
# chroma_data_path = '/content/drive/My Drive/GenerativeAI/MateAI/rag_assignment/ChromaDB_Data'

In [ ]:
# !zip -r db.zip ./db
!zip -r ChromaDB_Data.zip '/content/drive/My Drive/GenerativeAI/MateAI/rag_assignment/ChromaDB_Data'

In [ ]:
# To cleanup, you can delete the collection
#vectordb.delete_collection()
#vectordb.persist()

# delete the directory
#!rm -rf '/content/drive/My Drive/GenerativeAI/MateAI/rag_assignment/ChromaDB_Data'

## Starting again loading the db

restart the runtime

In [ ]:
#!unzip db.zip
#!unzip '/content/drive/My Drive/GenerativeAI/MateAI/rag_assignment/ChromaDB_Data.zip'

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive/')

In [ ]:
# Set the API key
#filepath = "/content/drive/My Drive/GenerativeAI/MateAI/"

#with open(filepath + "Jasper_OpenAI_API_Key.txt", "r") as f:
#  openai.api_key = ' '.join(f.readlines())

In [ ]:
#import os
#os.environ["OPENAI_API_KEY"] = openai.api_key

# <font color = green> Solution 2


***Model design***
- Multiple Files - PDFs
- LangChain
- Chroma Vector database
- Multi-doc retriever
- Instuctor Embeddings (Hugging Face)
- Local LLM: Flan-T5-XL
- No Open AI





## 1. <font color = red> Install and Import the Required Libraries

In [16]:
!pip -q install langchain tiktoken chromadb pypdf transformers InstructorEmbedding
!pip -q install accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.4 MB/s eta 0:00:00


In [17]:
!pip show langchain

Name: langchain
Version: 0.1.4
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, jsonpatch, langchain-community, langchain-core, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


## Setting up LangChain


In [18]:
import os

from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader


from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

## 2. <font color = red> Read, Process, and Chunk the PDF Files

In [19]:
# Load and process the text files
# loader = DirectoryLoader("/content/drive/My Drive/GenerativeAI/MateAI/rag_assignment/research_articles/", glob="./*.pdf", loader_cls=PyPDFLoader)
# documents = loader.load()
documents

[Document(page_content='Published as a conference paper at ICLR 2022\nTRAIN SHORT , TESTLONG : ATTENTION WITH LINEAR\nBIASES ENABLES INPUT LENGTH EXTRAPOLATION\nOﬁr Press1,2Noah A. Smith1,3Mike Lewis2\n1Paul G. Allen School of Computer Science & Engineering, University of Washington\n2Facebook AI Research\n3Allen Institute for AI\nofirp@cs.washington.edu\nABSTRACT\nSince the introduction of the transformer model by Vaswani et al. (2017), a funda-\nmental question has yet to be answered: how does a model achieve extrapolation\nat inference time for sequences that are longer than it saw during training? We ﬁrst\nshow that extrapolation can be enabled by simply changing the position represen-\ntation method, though we ﬁnd that current methods do not allow for efﬁcient ex-\ntrapolation. We therefore introduce a simpler and more efﬁcient position method,\nAttention with Linear Biases (ALiBi). ALiBi does not add positional embeddings\nto word embeddings; instead, it biases query-key attentio

In [20]:
len(documents)

346

## Load multiple and process documents

## Splitting, Chunking of text

In [21]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [22]:
len(texts)

1419

In [23]:
texts[2]

Document(page_content='sequences. More context, achieved by larger L, improves predictions at inference time. But longer\nsequences are more expensive to train on.2\nBefore transformers, RNN language models were trained on shorter- Lsequences and assumed to\ngeneralize to longer contexts at inference time (Mikolov et al., 2010; Mikolov & Zweig, 2012;\nZaremba et al., 2014). Vaswani et al. (2017), introducing the transformer, speculated that it “may\n[...] extrapolate to sequence lengths longer than the ones encountered during training.” We deﬁne\nextrapolation as a model’s ability to continue performing well as the number of input tokens during\nvalidation increases beyond the number of tokens on which the the model was trained. We ﬁnd\nthat transformer language models (LMs) that use sinusoidal position embeddings have very weak\nextrapolation abilities; see Figure 1.\nWe demonstrate that this failure to extrapolate is caused by the position embedding method. As', metadata={'source': '

### Loading Instructor Embeddings

In [33]:
!pip install accelerate

In [38]:
import accelerate

In [34]:
!pip install bitsandbytes

In [37]:
import bitsandbytes

In [36]:
!pip install -i https://test.pypi.org/simple/ bitsandbytes

Looking in indexes: https://test.pypi.org/simple/


In [40]:
!pip install pytorch-pretrained-bert
import pytorch_pretrained_bert as ppb
assert 'bert-large-cased' in ppb.modeling.PRETRAINED_MODEL_ARCHIVE_MAP

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.6 MB/s eta 0:00:00


In [ ]:
import torch
import transformers
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline

tokenizer = LlamaTokenizer.from_pretrained("TheBloke/wizardLM-7B-HF")

model = LlamaForCausalLM.from_pretrained("TheBloke/wizardLM-7B-HF",
                                              load_in_8bit=False,
                                              device_map='auto',
                                              torch_dtype=torch.float16,
                                              low_cpu_mem_usage=True
                                              )

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
import torch

pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=512,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15
)

local_llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
# We check if the LLM is working properly by asking a simple question:
print(local_llm('What is the currency of India?'))

In [ ]:
# Loading the Instructor Embeddings from Hugging Face
# Using GPU -> "cuda"

from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
                                                      model_kwargs={"device": "cuda"})

### Creating a Chroma Vector Database


In [ ]:
# Define the path where chroma collections will be stored
chroma_data_path = '/content/drive/My Drive/GenerativeAI/MateAI/rag_assignment/ChromaDB_Data'

In [ ]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = chroma_data_path

## Here is the nmew embeddings being used
embedding = instructor_embeddings

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [ ]:
# persiste the db to disk
vectordb.persist()
vectordb = None

In [ ]:
# Now we can load the persisted database from disk, and use it as normal.
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding)

## 4. <font color = red> Semantic Search

In this section, we will perform a semantic search of a query in the collections embeddings to get several top semantically similar results.

In [ ]:
## Make a retriever
retriever = vectordb.as_retriever(search_kwargs={"k": 1})